System information (for reproducibility):

In [2]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 11 × Apple M3 Pro
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, westmere)
Threads: 1 default, 0 interactive, 1 GC (on 11 virtual cores)


Load packages:

In [3]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/GitHub/biostat-m257-2024-spring/hw5`
Precompiling project...
  ? Convex


Status `~/Documents/GitHub/biostat-m257-2024-spring/hw5/Project.toml`
  [1e616198] COSMO v0.8.9
  [61c947e1] Clarabel v0.8.1
⌃ [f65535da] Convex v0.15.3
  [a93c6f00] DataFrames v1.6.1
  [60bf3e95] GLPK v1.2.1
  [2e9cd046] Gurobi v1.2.3
  [87dc4568] HiGHS v1.9.0
  [b99e6be6] Hypatia v0.8.1
  [4076af6c] JuMP v1.22.1
  [67920dd8] KNITRO v0.14.2
  [b8f27783] MathOptInterface v1.29.0
  [1ec41992] MosekTools v0.15.1
  [2f354839] Pajarito v0.8.2
  [08abe8d2] PrettyTables v2.3.1
  [c946c3f1] SCS v2.0.0
  [3eaba693] StatsModels v0.7.3
Info Packages marked with ⌃ have new versions available and may be upgradable.


In this exercise, we practice using disciplined convex programming (SDP in particular) to solve optimal design problems.

## Introduction to optimal design

Consider a linear model
\begin{eqnarray*}
	y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad i = 1,\ldots, n,
\end{eqnarray*}
where $\epsilon_i$ are independent Gaussian noises with common variance $\sigma^2$. It is well known that the least squares estimate $\hat{\boldsymbol{\beta}}$ is unbiased and has covariance $\sigma^2 (\sum_{i=1}^n \mathbf{x}_i \mathbf{x}_i^T)^{-1}$. 

In **exact optimal design**, given total number of $n$ allowable experiments, we want to choose among a list of $m$ candidate design points $\{\mathbf{x}_1, \ldots, \mathbf{x}_m\}$ such that the covariance matrix is minimized in some sense. In mathematical terms, we want to find an integer vector $\mathbf{n} = (n_1, \ldots, n_m)$ such that $n_i \ge 0$, $\sum_{i=1}^m n_i = n$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m n_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

In **approximate optimal design**,  we want to find a probability vector $\mathbf{p} = (p_1, \ldots, p_m)$ such that $p_i \ge 0$, $\sum_{i=1}^m p_i = 1$, and the matrix $\mathbf{V} = \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1}$ is "small".

Commonly used optimal design criteria include:

- In **$D$-optimal design**, we minimize the determinant of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \det \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}

- In **$E$-optimal design**, we minimize the spectral norm, i.e., the maximum eigenvalue of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \lambda_{\text{max}} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.	
\end{eqnarray*}
Statistically we are minimizing the maximum variance of $\sum_{j=1}^p a_j \text{var}(\hat \beta_j)$ over all vectors $\mathbf{a}$ with unit norm.

- In **$A$-optimal design**, we minimize the trace of $\mathbf{V}$
\begin{eqnarray*}
	&\text{minimize}& \text{tr} \left( \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T \right)^{-1} \\
	&\text{subject to}& p_i \ge 0, \sum_{i=1}^m p_i = 1.
\end{eqnarray*}
Statistically we are minimizing the total variance $\sum_{j=1}^p \text{var}(\hat \beta_j)$.

## Q1 (10 pts) 3x4 factorial design

A drug company ask you to help design a two factor clinical trial, in which treatment A has three levels (A1, A2, and A3) and treatment B has four levels (B1, B2, B3, and B4). Drug company also tells you that the treatment combination A3:B4 has undesirable side effects so we ignore this design point. 

Using dummy coding with A1 and B1 as the baseline levels, find the matrix $C$ with each row a unique design point.

### Solution

In [6]:
using DataFrames, MosekTools, MathOptInterface, LinearAlgebra, JuMP, Pajarito, Gurobi, Convex

In [127]:
treatment = ["A1:B1", "A2:B1", "A3:B1", "A1:B2", "A2:B2", 
                         "A3:B2", "A1:B3", "A2:B3", "A3:B3", "A1:B4", "A2:B4"]
A2 = [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
A3 = [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
B2 = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
B3 = [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]
B4 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]

df = DataFrame(treatment = treatment, intercept = ones(11),
               A2 = A2, A3 = A3, B2 = B2, B3 = B3, B4 = B4)
df

Row,treatment,intercept,A2,A3,B2,B3,B4
,String,Float64,Int64,Int64,Int64,Int64,Int64
1,A1:B1,1.0,0,0,0,0,0
2,A2:B1,1.0,1,0,0,0,0
3,A3:B1,1.0,0,1,0,0,0
4,A1:B2,1.0,0,0,1,0,0
5,A2:B2,1.0,1,0,1,0,0
6,A3:B2,1.0,0,1,1,0,0
7,A1:B3,1.0,0,0,0,1,0
8,A2:B3,1.0,1,0,0,1,0
9,A3:B3,1.0,0,1,0,1,0


## Q2 (30 pts) Find approximate optimal designs

Using semidefinite programming (SDP) software to find the approximate D-, E-, and A-optimal designs for this clinical trial.

Hint: This is what I got (using Hypatia solver), which may or may not be correct.

```
Approximate Optimal Design
┌───────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │ D_opt_n │ E_opt_n │ A_opt_n │
│    String │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│      A1B1 │   0.082 │   0.271 │   0.200 │   8.000 │  27.000 │  20.000 │
│      A2B1 │   0.082 │   0.150 │   0.101 │   8.000 │  15.000 │  10.000 │
│      A3B1 │   0.097 │   0.118 │   0.104 │  10.000 │  12.000 │  10.000 │
│      A1B2 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B2 │   0.082 │   0.037 │   0.051 │   8.000 │   4.000 │   5.000 │
│      A3B2 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   0.082 │   0.058 │   0.086 │   8.000 │   6.000 │   9.000 │
│      A2B3 │   0.082 │   0.037 │   0.051 │   8.000 │   3.000 │   5.000 │
│      A3B3 │   0.097 │   0.059 │   0.068 │  10.000 │   6.000 │   7.000 │
│      A1B4 │   0.109 │   0.079 │   0.106 │  11.000 │   8.000 │  10.000 │
│      A2B4 │   0.109 │   0.075 │   0.080 │  11.000 │   7.000 │   8.000 │
│       Obj │   8.987 │  13.000 │  38.925 │   8.988 │  13.028 │  38.946 │
└───────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘
```

In [177]:
X = hcat(ones(11), A2, A3, B2, B3, B4)
X

11×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  1.0  0.0  1.0  0.0  0.0
 1.0  0.0  1.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0  1.0  0.0
 1.0  0.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  0.0  0.0  0.0  1.0

In [160]:
#D-opt
p = Variable(size(X, 1))
D_problem = maximize(logdet(transpose(X) * Diagonal(p) * X)) # objective
D_problem.constraints += sum(p) == 1; # constraint
D_problem.constraints += p >= 0;
D_problem

maximize
└─ logdet (concave; real)
   └─ * (affine; real)
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      └─ 11×6 Matrix{Float64}
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 174…097)
│  └─ 1
└─ >= constraint (affine)
   ├─ 11-element real variable (id: 174…097)
   └─ 0

status: `solve!` not called yet

In [161]:
using MosekTools, MathOptInterface
const MOI = MathOptInterface

solver = Mosek.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("LOG"), 0)

solve!(D_problem, solver)
D_opt = vec(round.(p.value, digits = 3))
D_problem.status, D_problem.optval, D_opt

(MathOptInterface.OPTIMAL, -8.986831543894127, [0.082, 0.082, 0.097, 0.082, 0.082, 0.097, 0.082, 0.082, 0.097, 0.109, 0.109])

In [162]:
D_opt_n = vec(round.(D_opt .* 100))

11-element Vector{Float64}:
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
 11.0
 11.0

For E-opt, note that minimize $\lambda_{max}\left(\sum_{i=1}^mp_ix_ix_i^T \right)^{-1} = $ minimize $1/\lambda_{min}\left(\sum_{i=1}^mp_ix_ix_i^T \right) = $ maximize $\lambda_{min}\left(\sum_{i=1}^mp_ix_ix_i^T \right)$

In [163]:
#E-opt
p = Variable(size(X, 1))
E_problem = maximize(eigmin((transpose(X) * Diagonal(p) * X))) # objective
E_problem.constraints += sum(p) == 1; # constraint
E_problem.constraints += p >= 0;
E_problem

maximize
└─ eigmin (concave; real)
   └─ * (affine; real)
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      └─ 11×6 Matrix{Float64}
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 150…253)
│  └─ 1
└─ >= constraint (affine)
   ├─ 11-element real variable (id: 150…253)
   └─ 0

status: `solve!` not called yet

In [164]:
using MosekTools, MathOptInterface
const MOI = MathOptInterface

solver = Mosek.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("LOG"), 0)

solve!(E_problem, solver)
E_opt = vec(round.(p.value, digits = 3))
E_problem.status, E_problem.optval, E_opt

(MathOptInterface.OPTIMAL, 0.07692307672536335, [0.271, 0.156, 0.111, 0.057, 0.042, 0.056, 0.057, 0.042, 0.056, 0.081, 0.073])

In [165]:
E_opt_n = vec(round.(E_opt .* 100))

11-element Vector{Float64}:
 27.0
 16.0
 11.0
  6.0
  4.0
  6.0
  6.0
  4.0
  6.0
  8.0
  7.0

For A-opt, as we are not able to derive the $tr(V) =$ $tr\left(\sum_{i=1}^mp_ix_ix_i^T \right)^{-1}$ directly, I consider to use Schur's complement instead. Note that the Schur complement for a matrix $\begin{bmatrix}A & B \\ C & D \end{bmatrix}$ is $A - BD^{-1}C$ with respect to $D$ if $D$ is invertible, and this matrix is positive-semi definite is equivalent to $A$ is PSD and $A - BD^{-1}C$ is also PSD. Hence, I construct a new matrix $M = \begin{bmatrix}Y & I \\ I & V^{-1} \end{bmatrix}$, where $V^{-1} = \sum_{i=1}^mp_ix_ix_i^T$. Now if we add a constraint on that as $M$ is PSD, we can derive that $Y - V$ is PSD, which can be used to infer that $tr(Y) \geq tr(V)$. Therefore, if we can minimize $tr(Y)$, then $tr(V)$ will also be minimized.

In [166]:
#A-opt
p = Variable(size(X, 1))
Y = Semidefinite(size(X, 2))
schur_matrix = [Y I(size(X, 2)); I(size(X, 2)) (transpose(X) * Diagonal(p) * X)]
A_problem = minimize(tr(Y)) # objective
A_problem.constraints += sum(p) == 1; # constraint
A_problem.constraints += p >= 0;
A_problem.constraints += schur_matrix in :SDP;
A_problem

minimize
└─ sum (affine; real)
   └─ diag (affine; real)
      └─ 6×6 real variable (id: 181…195)
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 120…727)
│  └─ 1
├─ >= constraint (affine)
│  ├─ 11-element real variable (id: 120…727)
│  └─ 0
└─ sdp constraint (affine)
   └─ transpose (affine; real)
      └─ hcat (affine; real)
         ├─ …
         └─ …

status: `solve!` not called yet

In [167]:
using MosekTools, MathOptInterface
const MOI = MathOptInterface

solver = Mosek.Optimizer()
MOI.set(solver, MOI.RawOptimizerAttribute("LOG"), 0)

solve!(A_problem, solver)
A_opt = vec(round.(p.value, digits = 3))
A_problem.status, A_problem.optval, A_opt

(MathOptInterface.OPTIMAL, 38.92481411736323, [0.2, 0.101, 0.104, 0.086, 0.051, 0.068, 0.086, 0.051, 0.068, 0.106, 0.08])

In [168]:
A_opt_n = vec(round.(A_opt .* 100))

11-element Vector{Float64}:
 20.0
 10.0
 10.0
  9.0
  5.0
  7.0
  9.0
  5.0
  7.0
 11.0
  8.0

### Summary table for Q2

In [174]:
df_q2 = DataFrame(treatment = treatment, 
               D_opt = D_opt, E_opt = E_opt, A_opt = A_opt,
               D_opt_n = D_opt_n, E_opt_n = E_opt_n, A_opt_n = A_opt_n)
println(df_q2)
println(round(D_problem.optval, digits = 3), " ", round(E_problem.optval, digits = 3), 
    " ", round(A_problem.optval, digits = 3))

11×7 DataFrame
 Row │ treatment  D_opt    E_opt    A_opt    D_opt_n  E_opt_n  A_opt_n 
     │ String     Float64  Float64  Float64  Float64  Float64  Float64 
─────┼─────────────────────────────────────────────────────────────────
   1 │ A1:B1        0.082    0.271    0.2        8.0     27.0     20.0
   2 │ A2:B1        0.082    0.156    0.101      8.0     16.0     10.0
   3 │ A3:B1        0.097    0.111    0.104     10.0     11.0     10.0
   4 │ A1:B2        0.082    0.057    0.086      8.0      6.0      9.0
   5 │ A2:B2        0.082    0.042    0.051      8.0      4.0      5.0
   6 │ A3:B2        0.097    0.056    0.068     10.0      6.0      7.0
   7 │ A1:B3        0.082    0.057    0.086      8.0      6.0      9.0
   8 │ A2:B3        0.082    0.042    0.051      8.0      4.0      5.0
   9 │ A3:B3        0.097    0.056    0.068     10.0      6.0      7.0
  10 │ A1:B4        0.109    0.081    0.106     11.0      8.0     11.0
  11 │ A2:B4        0.109    0.073    0.08      11.0      7

## Q3 (30 pts) Find exact optimal designs

Using mixed-integer semidefinite programming (SDP) software to find the exact D-, E-, and A-optimal designs for this clinical trial **with $n=100$**.

Hint: This is what I got using Pajarito with Gurobi as the `oa_solver` and Mosek as the `conic_solver`.

```
Exact Optimal Design
┌───────────┬─────────┬─────────┬─────────┐
│ design_pt │   D_opt │   E_opt │   A_opt │
│    String │ Float64 │ Float64 │ Float64 │
├───────────┼─────────┼─────────┼─────────┤
│      A1B1 │   8.000 │  34.000 │  20.000 │
│      A2B1 │   8.000 │  10.000 │  10.000 │
│      A3B1 │  10.000 │  10.000 │  10.000 │
│      A1B2 │   8.000 │   9.000 │   9.000 │
│      A2B2 │   8.000 │   0.000 │   5.000 │
│      A3B2 │  10.000 │   6.000 │   7.000 │
│      A1B3 │   8.000 │   8.000 │   9.000 │
│      A2B3 │   8.000 │   4.000 │   5.000 │
│      A3B3 │  10.000 │   4.000 │   7.000 │
│      A1B4 │  11.000 │   9.000 │  10.000 │
│      A2B4 │  11.000 │   6.000 │   8.000 │
│       Obj │   8.988 │  13.011 │  38.946 │
└───────────┴─────────┴─────────┴─────────┘
```

In [183]:
X = Int64.(X)
X

11×6 Matrix{Int64}:
 1  0  0  0  0  0
 1  1  0  0  0  0
 1  0  1  0  0  0
 1  0  0  1  0  0
 1  1  0  1  0  0
 1  0  1  1  0  0
 1  0  0  0  1  0
 1  1  0  0  1  0
 1  0  1  0  1  0
 1  0  0  0  0  1
 1  1  0  0  0  1

In [190]:
# define problem
N = Variable(size(X, 1), :Int)
# D-opt
D_problem = maximize(logdet(transpose(X) * Diagonal(N) * X))# objective
D_problem.constraints += sum(N) == 100; # constraint
D_problem.constraints += N >= 0; # constraint
D_problem

maximize
└─ logdet (concave; real)
   └─ * (affine; real)
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      └─ 11×6 Matrix{Int64}
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 159…800)
│  └─ 100
└─ >= constraint (affine)
   ├─ 11-element real variable (id: 159…800)
   └─ 0

status: `solve!` not called yet

In [191]:
# define solver
misdp_solver = optimizer_with_attributes(
        Pajarito.Optimizer,
    "oa_solver" => optimizer_with_attributes(
        Gurobi.Optimizer,
        MOI.Silent() => true),
    "conic_solver" => optimizer_with_attributes(
        MosekTools.Optimizer, 
        MOI.Silent() => true),
)
set_attribute(misdp_solver, "time_limit", 60)
MOI.set(misdp_solver, MOI.RawOptimizerAttribute("verbose"), 0)
solve!(D_problem, misdp_solver)
# Check the status, optimal value, and minimizer of the problem
D_problem.status, D_problem.optval, N.value

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-22


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


(MathOptInterface.OPTIMAL, 18.64208131074323, [9.0; 8.0; … ; 11.0; 11.0;;])

In [192]:
D_opt_n = vec(N.value)

11-element Vector{Float64}:
  9.0
  8.0
  9.0
  8.0
  8.0
 10.0
  8.0
  8.0
 10.0
 11.0
 11.0

In [194]:
#E-opt
N = Variable(size(X, 1), :Int)
E_problem = maximize(eigmin((transpose(X) * Diagonal(N) * X))) # objective
E_problem.constraints += sum(N) == 100; # constraint
E_problem.constraints += N >= 0;
E_problem

maximize
└─ eigmin (concave; real)
   └─ * (affine; real)
      ├─ * (affine; real)
      │  ├─ …
      │  └─ …
      └─ 11×6 Matrix{Int64}
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 947…723)
│  └─ 100
└─ >= constraint (affine)
   ├─ 11-element real variable (id: 947…723)
   └─ 0

status: `solve!` not called yet

In [195]:
const MOI = MathOptInterface

MOI.set(misdp_solver, MOI.RawOptimizerAttribute("verbose"), 0)
solve!(E_problem, misdp_solver)
# Check the status, optimal value, and minimizer of the problem
E_problem.status, E_problem.optval, N.value

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-22
new incumbent
new incumbent
new incumbent


(MathOptInterface.OPTIMAL, 7.685749750491152, [29.0; 15.0; … ; 7.0; 8.0;;])

In [196]:
E_opt_n = vec(N.value)

11-element Vector{Float64}:
 29.0
 15.0
 10.0
  6.0
  6.0
  4.0
  8.0
  1.0
  6.0
  7.0
  8.0

In [212]:
#A-opt
N = Variable(size(X, 1), :Int)
Y = Semidefinite(size(X, 2))
schur_matrix = [Y I(size(X, 2)); I(size(X, 2)) (transpose(X) * Diagonal(N) * X)]
A_problem = minimize(tr(Y)) # objective
A_problem.constraints += sum(N) == 100; # constraint
A_problem.constraints += N >= 0;
A_problem.constraints += schur_matrix in :SDP;
A_problem

minimize
└─ sum (affine; real)
   └─ diag (affine; real)
      └─ 6×6 real variable (id: 311…845)
subject to
├─ == constraint (affine)
│  ├─ sum (affine; real)
│  │  └─ 11-element real variable (id: 171…965)
│  └─ 100
├─ >= constraint (affine)
│  ├─ 11-element real variable (id: 171…965)
│  └─ 0
└─ sdp constraint (affine)
   └─ transpose (affine; real)
      └─ hcat (affine; real)
         ├─ …
         └─ …

status: `solve!` not called yet

In [213]:
const MOI = MathOptInterface

MOI.set(misdp_solver, MOI.RawOptimizerAttribute("verbose"), 0)
solve!(A_problem, misdp_solver)
# Check the status, optimal value, and minimizer of the problem
A_problem.status, A_problem.optval, N.value

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-22


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


norm of dual is 1.545274139962598e14


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: integer variable solution is not integral to tolerance tol_feas
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:441
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: integer variable solution is not integral to tolerance tol_feas
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:441
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


new incumbent


┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396
┌ Warning: continuous subproblem status SLOW_PROGRESS is not handled
└ @ Pajarito ~/.julia/packages/Pajarito/gSNvz/src/algorithms.jl:396


new incumbent


(MathOptInterface.OPTIMAL, 0.38945777084334887, [20.0; 10.0; … ; 10.0; 8.0;;])

In [214]:
A_opt_n = vec(N.value)

11-element Vector{Float64}:
 20.0
 10.0
 10.0
  9.0
  5.0
  7.0
  9.0
  5.0
  7.0
 10.0
  8.0

### Summary table for Q3

In [217]:
df = DataFrame(treatment = treatment, 
               D_opt_n = Int64.(D_opt_n), E_opt_n = Int64.(E_opt_n), A_opt_n = Int64.(A_opt_n))
df

Row,treatment,D_opt_n,E_opt_n,A_opt_n
,String,Int64,Int64,Int64
1,A1:B1,9,29,20
2,A2:B1,8,15,10
3,A3:B1,9,10,10
4,A1:B2,8,6,9
5,A2:B2,8,6,5
6,A3:B2,10,4,7
7,A1:B3,8,8,9
8,A2:B3,8,1,5
9,A3:B3,10,6,7


## Q4 (30 bonus points) Optimal design with nuisance parameters

Suppose the regression coefficients of linear model $\boldsymbol{\beta}$ is partitioned as $\boldsymbol{\beta} = (\boldsymbol{\beta}_0^T, \boldsymbol{\beta}_1^T)^T$, where $\boldsymbol{\beta}_0$ are nuisance parameters and $\boldsymbol{\beta}_1$ are parameters of primary interest. Given an approximate design $\mathbf{p} = (p_1, \ldots, p_m)$, let the information matrix be partitioned accordingly
$$
\mathbf{I}(\mathbf{p}) = \sum_{i=1}^m p_i \mathbf{x}_i \mathbf{x}_i^T =  \begin{pmatrix}
\mathbf{I}_{00} & \mathbf{I}_{01} \\
\mathbf{I}_{10} & \mathbf{I}_{11}
\end{pmatrix}.
$$
Then the information matrix for $\boldsymbol{\beta}_1$ adjusted for nuisance parameter $\boldsymbol{\beta}_0$ is
$$
\mathbf{I}_{1 \mid 0}(\mathbf{p}) = \mathbf{I}_{11} - \mathbf{I}_{10} \mathbf{I}_{00}^{-1} \mathbf{I}_{01}.
$$

Revisiting the 3x4 factorial design problem in Q1, suppose the drug company only cares about the estimation of A treatment effects. Find the approximate D-, E-, and A-optimal designs.